In [494]:
import requests
res = requests.get("https://www.sigma-computer.com/search?search=laptop&submit_search=&route=product%2Fsearch")
res

<Response [200]>

In [498]:
from bs4 import BeautifulSoup
import requests

soup = BeautifulSoup(res.text, "html.parser")

Product_list_tags = soup.find_all("div", attrs={"class": "product-layout col-lg-15 col-md-4 col-sm-6 col-xs-12"})


In [499]:
len(Product_list_tags)

739

In [523]:
with open("Products.csv", mode="w", encoding="utf-8") as fd:
    fd.write("item_name,stock_status,price_new,price_old,Product_Link_Final\n")
    
    # Loop through each product
    for product_tag in Product_list_tags:
        try:
            item_name = product_tag.find("h4").text.replace('"', '""')
            
            price_new = product_tag.find("span", attrs={"class": "price-new"}).text
            price_old_tag = product_tag.find("span", attrs={"class": "price-old"})
            price_old = price_old_tag.text if price_old_tag else "N/A"
            product_link_tag = product_tag.find("a")
            product_link_partial = product_link_tag.attrs['href'] if product_link_tag else "#"
            product_link_final = product_link_partial if product_link_partial.startswith("http") else f"https://www.sigma-computer.com/{product_link_partial}"
            
            stock_status = "Unknown"
            stock_tag_instock = product_tag.find("span", attrs={"class": "stock"})
            stock_tag_outstock = product_tag.find("span", attrs={"class": "stock_N"})
            
            if stock_tag_instock and stock_tag_instock.text.strip() == "In Stock":
                stock_status = "In Stock"
            elif stock_tag_outstock and stock_tag_outstock.text.strip() == "Out Of Stock":
                stock_status = "Out Of Stock"
            
            fd.write(f"\"{item_name}\",\"{stock_status}\",\"{price_new}\",\"{price_old}\",\"{product_link_final}\"\n")
        except Exception as e:
            print(f"Error processing product: {e}")
